# Plan
- Mix different noise recording + sentence recording as noises
- Sine wave at varying frequency as the signal 
    - harmonic random weighting
    - custom filter later on 
- varying mixing ratio over time
